In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from torchvision.models import resnet18, ResNet18_Weights
import torch
import PIL

In [2]:
import numpy as np
from PIL import Image

DATASET_PATH = '/kaggle/input/dog-emotion/Dog Emotion/'

data = {'img': [], 'emotion': []}

for emotion in ['angry', 'happy', 'relaxed', 'sad']:
    for dirname, _, filenames in os.walk(DATASET_PATH + emotion + '/'):
        for filename in filenames:
            data['img'].append(os.path.join(dirname, filename))
            data['emotion'].append(emotion)

df = pd.DataFrame.from_dict(data)

In [3]:
df

,img,emotion
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry
...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad


In [4]:
emotion_labels_map = {}

unique_emotions = df['emotion'].unique()
for i in range(len(unique_emotions)):
    emotion_labels_map[unique_emotions[i]] = i

In [5]:
df['emotion_label'] = df['emotion'].apply(lambda x: emotion_labels_map[x])
df

,img,emotion,emotion_label
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry,0
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry,0
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry,0
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry,0
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry,0
...,...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad,3
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad,3
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad,3
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad,3


In [6]:
from torch.utils.data import DataLoader, Dataset

class Dataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = PIL.Image.open(self.data.loc[idx, "img"]).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, "emotion_label"])
        return image, label

In [7]:
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["img"], df["emotion_label"], random_state=42)

X_train.index = np.arange(len(X_train))
y_train.index = np.arange(len(y_train))
X_test.index = np.arange(len(X_test))
y_test.index = np.arange(len(y_test))

In [34]:
transform_train = transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(224),  # Random crop with resize to 224x224
    v2.RandomHorizontalFlip(p=0.5),  # Horizontal flip with 50% probability
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color adjustments
    v2.RandomRotation(15),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [35]:
train_ds = Dataset(pd.concat([X_train, y_train], axis=1), transform_train)
test_ds = Dataset(pd.concat([X_test, y_test], axis=1), transform_test)


In [36]:
import torch.nn as nn
import torch.optim as optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(in_features=512, out_features=4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 10
batch_size = 32
learning_rate = 3e-4
weight_decay = 0.0

train_loader = DataLoader(train_ds, batch_size=batch_size, drop_last=True)
test_loader = DataLoader(test_ds, batch_size=batch_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'Loss: {running_loss/100:.4f}, '
                  f'Accuracy: {100 * correct/total:.2f}%')
            running_loss = 0.0
    
print('Training complete')
    

Epoch [1/10], Step [10/93], Loss: 0.1359, Accuracy: 39.69%
Epoch [1/10], Step [20/93], Loss: 0.1186, Accuracy: 45.16%
Epoch [1/10], Step [30/93], Loss: 0.1195, Accuracy: 46.46%
Epoch [1/10], Step [40/93], Loss: 0.1044, Accuracy: 48.91%
Epoch [1/10], Step [50/93], Loss: 0.1070, Accuracy: 49.62%
Epoch [1/10], Step [60/93], Loss: 0.1026, Accuracy: 50.57%
Epoch [1/10], Step [70/93], Loss: 0.1039, Accuracy: 51.25%
Epoch [1/10], Step [80/93], Loss: 0.0986, Accuracy: 52.38%
Epoch [1/10], Step [90/93], Loss: 0.0891, Accuracy: 53.26%
Epoch [2/10], Step [10/93], Loss: 0.0864, Accuracy: 63.12%
Epoch [2/10], Step [20/93], Loss: 0.0865, Accuracy: 63.44%
Epoch [2/10], Step [30/93], Loss: 0.0910, Accuracy: 62.29%
Epoch [2/10], Step [40/93], Loss: 0.0867, Accuracy: 63.12%
Epoch [2/10], Step [50/93], Loss: 0.0866, Accuracy: 63.38%
Epoch [2/10], Step [60/93], Loss: 0.0867, Accuracy: 63.54%
Epoch [2/10], Step [70/93], Loss: 0.0907, Accuracy: 63.12%
Epoch [2/10], Step [80/93], Loss: 0.0788, Accuracy: 63.4

In [37]:
model.eval()

predictions = []
true_labels = []

for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    predictions += predicted.cpu().tolist()
    true_labels += labels.cpu().tolist()

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(predictions, true_labels)

0.815